In [4]:
import geopandas as gpd
from shapely.geometry import Point, shape, mapping, box, Polygon
import rasterio.mask
from rasterio.features import rasterize
import os
import csv
import rasterio
import time
import argparse
import shutil
import cv2
import numpy as np
from scipy.ndimage import rotate
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
import math
import matplotlib.pyplot as plt
import pdb
from rasterio.transform import rowcol
from math import atan2, degrees
from collections import defaultdict
import re
from rasterio.mask import geometry_mask
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
import affine
from shapely.geometry import mapping
from shapely.ops import transform as shapely_transform
import math
import concurrent.futures
from pathlib import Path
import pandas as pd

In [5]:
def crop_black_white_pixels(rotated_bands, plot_no, date, location, sensor, threshold=0.0, border=5):
    x_min, x_max, y_min, y_max = float('inf'), 0, float('inf'), 0
    has_valid_band = False

    for band in rotated_bands:
        nonzero_indices = np.where(band > threshold)
        if nonzero_indices[0].size > 0:
            has_valid_band = True
            x_min = min(x_min, np.min(nonzero_indices[1]))
            x_max = max(x_max, np.max(nonzero_indices[1]))
            y_min = min(y_min, np.min(nonzero_indices[0]))
            y_max = max(y_max, np.max(nonzero_indices[0]))

    if not has_valid_band:
        #cropped_bands = np.array([])
        print(f"No valid non-zero pixels found for plot {plot_no}. Check image and threshold.")
        return [], has_valid_band

    x_min, x_max = max(x_min + border, 0), max(x_max - border, 0)
    y_min, y_max = max(y_min + border, 0), max(y_max - border, 0)

    cropped_bands = [band[y_min:y_max+1, x_min:x_max+1] for band in rotated_bands]
    h, w = cropped_bands[0].shape

    if h > w:
        cropped_bands = [cv2.rotate(band, cv2.ROTATE_90_CLOCKWISE) for band in cropped_bands]
        print(f"Rotated vertically aligned image for plot {plot_no} to horizontal")

    """
    for i, cropped_band in enumerate(cropped_bands):
        plt.figure()
        plt.imshow(cropped_band, cmap='gray')
        plt.title(f"Cropped Image - Band {i+1}")
        plt.colorbar()
        plt.savefig(f"plots/cropped_band_{i+1}_{date}_{location}_{plot_no}_{sensor}.png")
        plt.close()
        break
    """
    return cropped_bands, has_valid_band

def get_longest_edge_angle(coordinates, out_transform):
    points = np.array(coordinates[0])

    def geo_to_pixel(x, y, transform):
        col, row = ~transform * (x, y)
        return [int(col), int(row)]
    pixel_coords = [geo_to_pixel(x, y, out_transform) for x, y in points]
    pixel_coords = np.array(pixel_coords[:-1]) 
    centroid = np.mean(pixel_coords, axis=0)

    def angle_from_centroid(pixel_coords):
        return np.arctan2(pixel_coords[1] - centroid[1], pixel_coords[0] - centroid[0])
    pixel_coords = sorted(pixel_coords, key=angle_from_centroid)
    edge_lengths = [np.linalg.norm(pixel_coords[i] - pixel_coords[(i + 1) % 4]) for i in range(4)]
    width = (edge_lengths[0] + edge_lengths[2]) / 2
    height = (edge_lengths[1] + edge_lengths[3]) / 2

    try:
        slope = (pixel_coords[1][1] - pixel_coords[0][1]) / (pixel_coords[1][0] - pixel_coords[0][0])
    except ZeroDivisionError:
        slope = float('inf')
    angle = math.atan(slope) * 180 / math.pi if slope != float('inf') else 90.0
    try:
        slope_vert = (pixel_coords[3][1] - pixel_coords[0][1]) / (pixel_coords[3][0] - pixel_coords[0][0])
    except ZeroDivisionError:
        slope_vert = float('inf')
    angle_vert = math.atan(slope_vert) * 180 / math.pi if slope_vert != float('inf') else 90.0
    if angle_vert < 0:
        angle2 = 90 + angle_vert
    else:
        angle2 = angle_vert
    return angle2 if height > width else angle


def _get_longest_edge_angle_from_geom(geom):
    """
    Calculates the normalized angle of a polygon's primary orientation
    by ensuring the final angle is always between -90 and +90 degrees.
    """
    mrr = geom.minimum_rotated_rectangle
    points = list(mrr.exterior.coords)

    if not points:
        return 0.0

    max_len_sq = 0
    longest_segment = (0, 0, 0, 0)

    for i in range(len(points) - 1):
        x1, y1 = points[i]
        x2, y2 = points[i+1]
        len_sq = (x2 - x1)**2 + (y2 - y1)**2
        if len_sq > max_len_sq:
            max_len_sq = len_sq
            longest_segment = (x1, y1, x2, y2)

    p1x, p1y, p2x, p2y = longest_segment
    angle_rad = math.atan2(p2y - p1y, p2x - p1x)
    angle_deg = math.degrees(angle_rad)

    # Normalize the angle to be between -90 and 90 degrees.
    # This corrects for 180-degree flips in direction.
    if angle_deg > 90.0:
        angle_deg -= 180.0
    elif angle_deg < -90.0:
        angle_deg += 180.0

    return angle_deg

In [3]:
def get_plot_tile_geospatial_old(row, image_path, plot_no, date, location, sensor):
    """
    The version that was producing a result with only a small (5cm) offset.
    Uses the bounding box center as the pivot point.
    """
    with rasterio.open(image_path) as src:
        global crop_black_white_pixels

        # Use the buffered geometry for all calculations for consistency
        buffered_geometry = row["geometry"].buffer(5 * abs(src.transform.a))

        # 1. Get angle from the buffered geometry
        angle = _get_longest_edge_angle_from_geom(buffered_geometry)

        # 2. Define rotation using the BOUNDING BOX CENTER as the pivot
        bounds = buffered_geometry.bounds
        pivot_coords = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

        rot_around_center = affine.Affine.rotation(angle, pivot=pivot_coords)
        rotated_geom = shapely_transform(lambda x, y, z=None: rot_around_center * (x, y), buffered_geometry)
        dst_bounds = rotated_geom.bounds

        # 3. Build the destination transform
        buffer_px = 50
        pixel_size = src.res[0]
        top_left_x = dst_bounds[0] - (buffer_px * pixel_size)
        top_left_y = dst_bounds[3] + (buffer_px * pixel_size)

        dst_transform = (
            affine.Affine.translation(top_left_x, top_left_y) *
            affine.Affine.rotation(angle) *
            affine.Affine.scale(pixel_size, -pixel_size)
        )

        # 4. Calculate output dimensions
        dst_width = math.ceil(abs(dst_bounds[2] - dst_bounds[0]) / pixel_size) + (buffer_px * 2)
        dst_height = math.ceil(abs(dst_bounds[3] - dst_bounds[1]) / pixel_size) + (buffer_px * 2)

        # --- START DEBUGGING BLOCK ---
        debug_dir = "debug_shapefiles_om"
        os.makedirs(debug_dir, exist_ok=True)
        
        # Create a polygon representing the VRT canvas
        vrt_b = dst_transform * (0, 0)
        vrt_c = dst_transform * (dst_width, dst_height)
        vrt_canvas_geom = box(vrt_b[0], vrt_c[1], vrt_c[0], vrt_b[1])
        
        # Save the two key shapes to uniquely named files
        gpd.GeoDataFrame({'id': [plot_no]}, geometry=[buffered_geometry], crs=src.crs).to_file(os.path.join(debug_dir, f"plot_{plot_no}_1_buffered.shp"))
        gpd.GeoDataFrame({'id': [plot_no]}, geometry=[vrt_canvas_geom], crs=src.crs).to_file(os.path.join(debug_dir, f"plot_{plot_no}_2_vrt_canvas.shp"))
        # --- END DEBUGGING BLOCK ---

        # 5. Create the WarpedVRT
        with WarpedVRT(src,
                    crs=src.crs,
                    transform=dst_transform,
                    width=dst_width,
                    height=dst_height,
                    resampling=Resampling.bilinear,
                    nodata=src.nodata) as vrt:

            # 6. Mask from the VRT using the same buffered geometry
            shapes = [mapping(buffered_geometry)]
            try:
                rotated_image, out_transform = rasterio.mask.mask(
                    vrt,
                    shapes,
                    crop=True,
                    all_touched=True,
                    filled=True
                )
            except ValueError as e:
                print(f"Warning: Plot {plot_no} could not be masked. Error: {e}")
                return None, False, None

        # 7. Post-process
        cropped_image, has_valid_band = crop_black_white_pixels(
            list(rotated_image), plot_no, date, location, sensor
        )
        
        # 8. Convert to final numpy array
        final_cropped = np.array(cropped_image, dtype=src.dtypes[0])
        return plot_no,final_cropped, has_valid_band, out_transform